## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob

## Read-in

In [2]:
# Get today's date
today = datetime.now()

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Jul. 8th Jul. 15th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [8]:
# # Filter dates between. If only using data looking back one week, this shouldn't change the df
# df = df[(df['SOLD DATE'] >= '2024-06-10') & (df['SOLD DATE'] <= '2024-06-17')]

In [7]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
26,PAST SALE,2024-07-15,Condo/Co-op,250 NW 107th Ave #204,Miami,FL,33172.0,290000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/250-NW-107th-A...,MARMLS,A11584033,N,Y,25.771778,-80.369582
3,PAST SALE,2024-07-13,Condo/Co-op,2730 NE 4th St #106,Homestead,FL,33033.0,310000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Homestead/2730-NE-4t...,Beaches MLS,F10450863,N,Y,25.473871,-80.437073
1,PAST SALE,2024-07-12,Condo/Co-op,101 20th St Unit TH-E,Miami Beach,FL,33139.0,3900000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/101-20th...,MARMLS,A11623159,N,Y,25.795440,-80.128050
79,PAST SALE,2024-07-12,Condo/Co-op,7600 Fisher Island Dr #7654,Miami Beach,FL,33109.0,11825000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7600-Fis...,MARMLS,A11524666,N,Y,25.761225,-80.136382
39,PAST SALE,2024-07-12,Condo/Co-op,10370 SW 220th St #219,Cutler Bay,FL,33190.0,175000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Cutler-Bay/10370-SW-...,MARMLS,A11552290,N,Y,25.561532,-80.359622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,PAST SALE,2024-07-08,Condo/Co-op,6385 W 27th Ave Unit 23-3,Hialeah,FL,33016.0,269000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/6385-W-27th-...,MARMLS,A11564051,N,Y,25.880625,-80.336862
77,PAST SALE,2024-07-08,Condo/Co-op,13499 Biscayne Blvd #1103,North Miami,FL,33181.0,143000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/13499-Bi...,MARMLS,A11507427,N,Y,25.899600,-80.159691
11,PAST SALE,2024-07-08,Condo/Co-op,9423 Fontainebleau Blvd #201,Miami,FL,33172.0,317000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/9423-Fontaineb...,MARMLS,A11596545,N,Y,25.778719,-80.346794
12,PAST SALE,2024-07-08,Condo/Co-op,13621 Deering Bay Dr #1104,Coral Gables,FL,33158.0,2600000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/13621-D...,MARMLS,A11573369,N,Y,25.637681,-80.290989


In [8]:
df = df.sort_values(by='PRICE',ascending=False)
df_top_ten = df.head(10)

## Current Week's Values

In [9]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

113


In [10]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$821,936


In [11]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$563


In [12]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$92,878,749


In [13]:
# # Define the date range
# start_date = '2024-05-20'
# end_date = '2024-05-26'

# # Filter rows where 'SOLD DATE' is within the specified range
# second_week_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

## Previous Week's Values

In [13]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
118
Input Previous Week Condo Average Sales Price:
780998
Input Previous Week Condo Average PSF:
494
Input Previous Week Condo Sales total (ex: 198_000_000)
92_200_000


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [15]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [14]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61686/2105881173.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61686/2105881173.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')


## HTML Popup Formatter

In [15]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'RANK', 'COLOR'],
      dtype='object')

In [16]:
pd.set_option('display.max_columns',None)

In [17]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
79,PAST SALE,2024-07-12,Condo/Co-op,7600 Fisher Island Dr #7654,Miami Beach,FL,33109.0,11825000.0,3.0,4.5,7600 OCEANSIDE AT FISHER,3754.0,NaN,2001.0,NaN,3150.0,9404.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7600-Fis...,MARMLS,A11524666,N,Y,25.761225,-80.136382,1,orange


In [18]:
def popup_html(row):
    building_name = row['LOCATION']
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [19]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [20]:
m.save('index.html')

## Data snagger

In [21]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [22]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
79,PAST SALE,2024-07-12,Condo/Co-op,7600 Fisher Island Dr #7654,Miami Beach,FL,33109.0,11825000.0,3.0,4.5,7600 OCEANSIDE AT FISHER,3754.0,NaN,2001.0,NaN,3150.0,9404.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7600-Fis...,MARMLS,A11524666,N,Y,25.761225,-80.136382,1,orange


In [ ]:
# , {df_top_ten[df_top_ten['PRICE'].idxmax()]['ADDRESS']} | Price ${df_top_ten[df_top_ten['PRICE'].idxmax()]['PRICE']:,.0f}

In [23]:
df_top_ten['PRICE'].iloc[0]

11825000.0

In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [24]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_07152024


# CREATE TEMPLATE 

In [25]:
muni_set = set(df_top_ten['CITY'])

In [26]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61686/1479489112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']


In [27]:
df_top_ten.reset_index(inplace=True,drop=True)

In [28]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [29]:
top_sale

'7600 OCEANSIDE AT FISHER at 7600 Fisher Island Dr #7654 in Miami Beach'

In [30]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [34]:
# df_top_ten

## RESUME HERE

In [31]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} sales average from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: 7600 OCEANSIDE AT FISHER closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,800,000M to $11,825,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Key Biscayne, Coral Gables, Miami Beach, Bal Harbour, Miami, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 113 condo sales totaling $92,878,749 million from Jul. 8th to Jul. 15th. The previous week, brokers closed 118 condo sales totaling $92,200,000.

Last week’s units sold for an average of $821,936, higher than the $780,998 sales average from the

In [32]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [33]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [34]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf")

print(story_checklist)


HED: 7600 OCEANSIDE AT FISHER closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,800,000M to $11,825,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Key Biscayne, Coral Gables, Miami Beach, Bal Harbour, Miami, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 113 condo sales totaling $92,878,749 million from Jul. 8th to Jul. 15th. The previous week, brokers closed 118 condo sales totaling $92,200,000.

Last week’s units sold for an average of $821,936, higher than the $780,998 sales average from the

In [35]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Miami-Beach/7600-Fisher-Is-Dr-33109/unit-7654/home/43269484


In [36]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/6800-Indian-Creek-Dr-33141/unit-1/home/185381555


In [37]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami-Beach/6799-Collins-Ave-33141/unit-1505/home/43396049


In [38]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Sunny-Isles-Beach/16485-Collins-Ave-33160/unit-1731/home/43247490


In [39]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/7600-Fisher-Is-Dr-33109/unit-7654/home/43269484


In [45]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Sunny-Isles-Beach/16485-Collins-Ave-33160/unit-1731/home/43247490


In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-07-12,Condo/Co-op,7600 Fisher Island Dr #7654,Miami Beach,FL,33109.0,11825000.0,3.0,4.5,7600 OCEANSIDE AT FISHER,3754.0,NaN,2001.0,NaN,3150.0,9404.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7600-Fis...,MARMLS,A11524666,N,Y,25.761225,-80.136382,1,orange,7600 OCEANSIDE AT FISHER at 7600 Fisher Island...
1,PAST SALE,2024-07-09,Condo/Co-op,6800 Indian Creek Dr Ph 1,Miami Beach,FL,33141.0,5700000.0,3.0,3.5,MONACO YACHT CLUB RESIDENC,2539.0,NaN,2022.0,NaN,2245.0,5203.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6800-Ind...,MARMLS,A11493913,N,Y,25.852444,-80.122990,2,blue,MONACO YACHT CLUB RESIDENC at 6800 Indian Cree...
2,PAST SALE,2024-07-09,Condo/Co-op,445 Grand Bay Dr #908,Key Biscayne,FL,33149.0,4500000.0,3.0,4.5,GRAND BAY RESIDENCES COND,3660.0,NaN,1996.0,NaN,1230.0,3549.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/445-Gra...,MARMLS,A11515637,N,Y,25.692619,-80.158994,3,blue,GRAND BAY RESIDENCES COND at 445 Grand Bay Dr ...
3,PAST SALE,2024-07-12,Condo/Co-op,101 20th St Unit TH-E,Miami Beach,FL,33139.0,3900000.0,2.0,2.0,SETAI RESORT & RESIDENCES,1565.0,NaN,2004.0,NaN,2492.0,5505.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/101-20th...,MARMLS,A11623159,N,Y,25.795440,-80.128050,4,blue,SETAI RESORT & RESIDENCES at 101 20th St Unit ...
4,PAST SALE,2024-07-10,Condo/Co-op,10101 Collins Ave Unit 8B,Bal Harbour,FL,33154.0,3330000.0,2.0,3.5,THE PALACE AT BAL HARBOUR,3110.0,NaN,1994.0,NaN,1071.0,4184.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10101-Co...,MARMLS,A11592194,N,Y,25.892679,-80.122902,5,blue,THE PALACE AT BAL HARBOUR at 10101 Collins Ave...
5,PAST SALE,2024-07-08,Condo/Co-op,788 NE 23rd St #1201,Miami,FL,33137.0,3200000.0,3.0,4.5,ELYSEE CONDOMINIUM,2743.0,NaN,2021.0,NaN,1167.0,3554.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/788-NE-23rd-St...,MARMLS,A11567858,N,Y,25.799065,-80.185542,6,blue,ELYSEE CONDOMINIUM at 788 NE 23rd St #1201 in ...
6,PAST SALE,2024-07-08,Condo/Co-op,13621 Deering Bay Dr #1104,Coral Gables,FL,33158.0,2600000.0,4.0,4.5,MILANO AT DEERING BAY CON,3288.0,NaN,2003.0,NaN,791.0,3289.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/13621-D...,MARMLS,A11573369,N,Y,25.637681,-80.290989,7,blue,MILANO AT DEERING BAY CON at 13621 Deering Bay...
7,PAST SALE,2024-07-09,Condo/Co-op,488 NE 18th St #3512,Miami,FL,33132.0,1900000.0,3.0,4.0,ARIA ON THE BAY CONDO,2018.0,NaN,2018.0,NaN,942.0,4052.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/488-NE-18th-St...,MARMLS,A11489323,N,Y,25.793508,-80.186688,8,blue,ARIA ON THE BAY CONDO at 488 NE 18th St #3512 ...
8,PAST SALE,2024-07-11,Condo/Co-op,16485 Collins Ave #1731,Sunny Isles Beach,FL,33160.0,1875000.0,3.0,3.0,OCEANIA III CONDO,2430.0,NaN,1991.0,NaN,772.0,2091.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11606777,N,Y,25.929178,-80.121497,9,blue,OCEANIA III CONDO at 16485 Collins Ave #1731 i...
9,PAST SALE,2024-07-08,Condo/Co-op,6799 Collins Ave #1505,Miami Beach,FL,33141.0,1800000.0,2.0,2.0,SOUTH CARILLON BEACH COND,1500.0,NaN,2008.0,NaN,1200.0,4652.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6799-Col...,MARMLS,A11519559,N,Y,25.852725,-80.120046,10,blue,SOUTH CARILLON BEACH COND at 6799 Collins Ave ...


## Time on Market Calculator

In [43]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2023, 12, 4) ## List (Earlier) date
date2 = datetime(2024, 7, 9) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

218
